# LSTM: Sectionals_v3 Predictions

Train an LSTM to analyze a horse’s past performance sequence and predict the horse's race pattern that reflects its projected strength today.

### Target is distance_back

✅ This model, unless it proves predictive on a standalone apactity, will be fed into the CatBoost/YetiRank model as a feature.

⸻

🔍 How Accurate is Ranking by Sectional Data Alone?

🔥 The Good:

If your LSTM is trained on well-normalized, consistent historical speed signals: • Relative ordering can be quite meaningful • Especially in smaller fields or when you’re identifying top 3–4 finishers • Even if absolute values are off, ranking is often more robust

❄️ The Limitations: • LSTM alone won’t account for today’s track conditions, surface bias, jockey/trainer changes, distance, class jump/drop • It may favor horses that ran fast recently but are now outclassed or mispositioned

In [1]:
# Setup Environment
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
os.environ.pop("CUDA_VISIBLE_DEVICES", None)

# Setup Environment
import time
from optuna.importance import MeanDecreaseImpurityImportanceEvaluator
import os
import logging
import copy
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import optuna
from torch.optim.lr_scheduler import OneCycleLR, StepLR
import torch.nn as nn
import numpy as np
import optuna.visualization as viz
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import (to_date, date_format, lpad, concat_ws, collect_list, struct, 
                                   col, count, row_number, abs, unix_timestamp, mean, 
                                   when, lit, min as F_min, max as F_max , upper, trim,
                                   mean as F_mean, countDistinct, last, first, when)
from src.data_preprocessing.data_prep1.data_utils import initialize_environment
from src.data_preprocessing.data_prep1.data_loader import load_data_from_postgresql

In [3]:
spark, jdbc_url, jdbc_properties, parquet_dir, log_file = initialize_environment()

Spark session created successfully.


In [4]:
def gps_sql_queries():
    queries = {
#         "gps_horse": """
#             SELECT g.course_cd, g.race_date,g.race_number,
#             REGEXP_REPLACE(TRIM(UPPER(saddle_cloth_number)), '\s+$', '') AS saddle_cloth_number, time_stamp, 
#             longitude, latitude, speed, progress, stride_frequency, g.post_time, location,
#             re.axciskey, h.horse_id, re.official_fin, h.horse_name
#             FROM gpspoint g
#             JOIN results_entries re on g.course_cd = re.course_cd
#                 AND g.race_date = re.race_date
#                 AND g.race_number = re.race_number
#                 AND g.saddle_cloth_number = re.program_num
#             JOIN horse h on re.axciskey = h.axciskey
#             WHERE speed is not null
#             AND progress is not null
#             AND stride_frequency is not null
#             """,
        "sectionals": """
            --  A) Identify all horse_id's entered in upcoming races
            WITH upcoming_horses AS (
                SELECT DISTINCT h.horse_id
                FROM runners r
                JOIN horse h ON r.axciskey = h.axciskey
                WHERE r.race_date >= CURRENT_DATE
                  AND r.course_cd IN (
                      'CNL','SAR','PIM','TSA','BEL','MVR','TWO','KEE','TAM','TTP','TKD',
                      'ELP','PEN','HOU','DMR','TLS','AQU','MTH','TGP','TGG','CBY','LRL',
                      'TED','IND','TCD','TOP'
                  )
            )
            --  B) Retrieve all *historical* (past) sectionals for those horses
            SELECT
                s.course_cd,
                s.race_date,
                s.race_number,
                h.horse_id,
                REGEXP_REPLACE(TRIM(UPPER(s.saddle_cloth_number)), '\s+$', '') AS saddle_cloth_number,
                s.gate_name,
                s.gate_numeric,
                s.length_to_finish,
                s.sectional_time,
                s.running_time,
                s.distance_back,
                s.distance_ran,
                s.number_of_strides,
                s.post_time
            FROM sectionals s
            JOIN runners r
                ON s.course_cd = r.course_cd
               AND s.race_date = r.race_date
               AND s.race_number = r.race_number
               AND s.saddle_cloth_number = r.saddle_cloth_number
            JOIN horse h
                ON r.axciskey = h.axciskey
            -- Only pull past races:
            WHERE r.race_date < CURRENT_DATE
              -- Basic data quality checks
              AND s.length_to_finish IS NOT NULL
              AND s.sectional_time   IS NOT NULL
              AND s.running_time     IS NOT NULL
              AND s.distance_back    IS NOT NULL
              AND s.distance_ran     IS NOT NULL
              AND s.number_of_strides IS NOT NULL
              -- Restrict to the *same* set of courses you care about:
              AND r.course_cd IN (
                  'CNL','SAR','PIM','TSA','BEL','MVR','TWO','KEE','TAM','TTP','TKD',
                  'ELP','PEN','HOU','DMR','TLS','AQU','MTH','TGP','TGG','CBY','LRL',
                  'TED','IND','TCD','TOP'
              )
              -- Finally, only get rows for horses that have upcoming races
              AND h.horse_id IN (
                  SELECT horse_id FROM upcoming_horses
              )
            """
    }
    return queries


In [5]:
queries = gps_sql_queries()
dfs = load_data_from_postgresql(spark, jdbc_url, jdbc_properties, queries, parquet_dir)
        # Suppose we have a dictionary of queries
for name, df in dfs.items():
    logging.info(f"DataFrame '{name}' loaded. Schema:")
    df.printSchema()
    if name == "gps_horse":
        gps_horse_df = df
    elif name == "sectionals":
        sec_pred_df = df    
    else:
        logging.error(f"Unknown DataFrame name: {name}")
        continue


root
 |-- course_cd: string (nullable = true)
 |-- race_date: date (nullable = true)
 |-- race_number: integer (nullable = true)
 |-- horse_id: integer (nullable = true)
 |-- saddle_cloth_number: string (nullable = true)
 |-- gate_name: string (nullable = true)
 |-- gate_numeric: double (nullable = true)
 |-- length_to_finish: double (nullable = true)
 |-- sectional_time: double (nullable = true)
 |-- running_time: double (nullable = true)
 |-- distance_back: double (nullable = true)
 |-- distance_ran: double (nullable = true)
 |-- number_of_strides: double (nullable = true)
 |-- post_time: timestamp (nullable = true)



In [6]:

start_time = time.time()
# # gps_horse_df.write.mode("overwrite").parquet(f"{parquet_dir}/gps_horse_df")
sec_pred_df.write.mode("overwrite").parquet(f"{parquet_dir}/sec_pred_df")
logging.info(f"Data written to Parquet in {time.time() - start_time:.2f} seconds")
    

In [7]:
# gps_horse_df = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/gps_horse_df")
sec_pred_df = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/sec_pred_df")


In [8]:
sec_pred_df.count()

194332

## Create race_id

In [ ]:
# Convert race_date to a proper date format
sec_pred__df = sec_pred__df.withColumn("race_date", to_date(col("race_date")))

# Construct a unique race_id using course_cd, race_date, and race_number
sec_pred__df = sec_pred__df.withColumn(
    "race_id",
    concat_ws(
        "_",
        col("course_cd"),
        date_format(col("race_date"), "yyyyMMdd"),
        lpad(col("race_number").cast("string"), 2, "0")
    )
)


## Create gate_seq_num Column

### 1) Cast gate_numeric to a Numeric Type

In [ ]:
# For example, cast gate_numeric to double:
sectionals_df = sectionals_df.withColumn("gate_numeric", col("gate_numeric").cast("double"))

# Then (re)do your steps with the numeric column

In [ ]:

# Step 1: Filter out finish rows to compute max gate_numeric per race
non_finish_df = sectionals_df.filter(F.col("gate_name") != "finish")

# Define a window partitioning by race identifiers
race_window = Window.partitionBy("course_cd", "race_date", "race_number")

# Compute maximum gate_numeric for each race (ignoring finish rows)
max_gate_df = non_finish_df.groupBy("course_cd", "race_date", "race_number") \
    .agg(F.max("gate_numeric").alias("max_gate_numeric"))

# Step 2: Join the maximum gate value back to the original DataFrame
joined_df = sectionals_df.join(max_gate_df, on=["course_cd", "race_date", "race_number"], how="left")

# Step 3: Create the new column 'gate_seq_num'
# If the gate is 'finish', assign max_gate_numeric + 0.5, else keep gate_numeric.
sectionals_df = joined_df.withColumn(
    "gate_seq_num",
    F.when(F.col("gate_name") == "finish", F.col("max_gate_numeric") + F.lit(0.5))
     .otherwise(F.col("gate_numeric"))
)

## Sort

In [ ]:
sectionals_df.select("saddle_cloth_number", "gate_name", "gate_seq_num", "running_time") \
  .filter("race_id = 'TGP_20240818_09' AND saddle_cloth_number = 3") \
  .orderBy(F.asc("gate_seq_num")) \
  .show(50, truncate=False)

## Extract the Final Distance for Each Horse

In [ ]:

# A) Filter just the finish rows
finish_df = sectionals_df.filter(F.col("gate_name") == "finish")

# B) Select the final distance for that horse
#    We'll rename it to "final_dist" to avoid conflict
finish_dist_df = finish_df.select(
    "race_id",
    "horse_id",
    F.col("distance_back").alias("final_dist")
)

## Join This Final Distance Back to All Rows

In [ ]:
# Join on (race_id, horse_id) so every row for that horse
# gets the final distance, if it exists.
with_final_dist = sectionals_df.join(
    finish_dist_df,
    on=["race_id", "horse_id"],
    how="left"
)

## Filter Out the Finish Row

In [ ]:
partial_only_df = with_final_dist.filter(F.col("gate_name") != "finish")

## Rename final_dist to target

In [ ]:
partial_only_df = partial_only_df.withColumnRenamed("final_dist", "target")

In [ ]:
sectionals_df = partial_only_df

sectionals_df.printSchema()

In [ ]:
# PySpark query to filter and select specific columns
filtered_sectionals_df = sectionals_df.filter(col("race_id") == "TGP_20240818_09") \
    .select("course_cd", "race_date", "race_number", "saddle_cloth_number").distinct()

# Show the result
filtered_sectionals_df.show()

In [ ]:
# from pyspark.sql import functions as F

# # 1) First, get a DataFrame mapping each race_id to the unique horse count.
# horses_per_race_df = (
#     sectionals_df
#     .groupBy("race_id")
#     .agg(F.countDistinct("horse_id").alias("num_horses"))
# )

# # 2) Now group *that* result by num_horses to see how many races have that count.
# dist_df = (
#     horses_per_race_df
#     .groupBy("num_horses")
#     .count()
#     .orderBy("num_horses")
# )
# dist_df.show()

In [ ]:
# # PySpark query to filter and select specific columns
# filtered_sectionals_df = sectionals_df.filter(col("race_id") == "TGP_20240818_09") \
#     .select("course_cd", "race_date", "race_number", "saddle_cloth_number").distinct()

# # Show the result
# filtered_sectionals_df.show()

In [ ]:
# # Step 1: create a DataFrame of only the races that have >=4 horses
# valid_races_df = (
#     horses_per_race_df
#     .filter(F.col("num_horses") >= 4)
#     .select("race_id")
#     .distinct()
# )

# # Step 2: Join the original data with valid_races_df to keep only those race_ids
# sectionals_df = sectionals_df.join(valid_races_df, on="race_id", how="inner")


# Compute Sequence Lenghts

### Filter Rows with >=1 and <=38 Sequences

In [ ]:

# Step 1: Compute sequence lengths per horse per race
df_seq_len = sectionals_df.groupBy("race_id", "horse_id").agg(F.count("*").alias("seq_length"))

# Step 2: Filter sequences with length between 1 and 38
filtered_df = df_seq_len.filter((F.col("seq_length") >= 1) & (F.col("seq_length") <= 38))

# Step 3: Join the filtered sequence lengths back with the original data
sectionals_df = sectionals_df.join(filtered_df, on=["race_id", "horse_id"], how="inner")

### Plot Sequence Lengths

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_seq_len is a DataFrame with a 'seq_length' column
# If df_seq_len is a PySpark DataFrame, convert it to Pandas
df_seq_len_pd = df_seq_len.toPandas()

# Plotting the histogram
plt.figure(figsize=(10, 6))
sns.histplot(df_seq_len_pd['seq_length'], bins=50, kde=True)
plt.title('Distribution of Sequence Lengths')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

## Padding Sequences to a Fixed Length

###  Step 1: Convert to Sequence Format

In [ ]:
sectionals_df.printSchema()

## Create segment_ordinal

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

w = Window.partitionBy("race_id", "horse_id").orderBy(F.asc("gate_seq_num"))

sectionals_df = sectionals_df.withColumn(
    "segment_ordinal",
    F.row_number().over(w)   # 1-based index for each call in ascending order
)

## Make Sequences

In [ ]:
features = [
    "length_to_finish",
    "sectional_time",
    "running_time",
    "distance_back",
    "distance_ran",
    "number_of_strides"
]

In [ ]:
from pyspark.sql.functions import struct, collect_list

df_features = sectionals_df.select(
    "race_id",
    "horse_id",
    "segment_ordinal",
    F.struct(*[F.col(f) for f in features]).alias("feature_struct")
)

# Window to collect all rows (time steps) for each (race_id, horse_id).
collect_w = Window.partitionBy("race_id", "horse_id").orderBy("segment_ordinal") \
                  .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_features = df_features.withColumn(
    "sequence",
    collect_list("feature_struct").over(collect_w)
)

# Now each row for that horse has a "sequence", but repeated for each time step.
# We only want the final row for each horse, i.e. the row with max(segment_ordinal).
df_features = df_features.withColumn(
    "rn",
    F.row_number().over(
        Window.partitionBy("race_id", "horse_id").orderBy(F.desc("segment_ordinal"))
    )
)

# Keep only rn == 1 (the row with the largest segment_ordinal => entire sequence):
df_features = df_features.filter(F.col("rn") == 1)

# Now df_features has columns: [race_id, horse_id, sequence], one row per horse.
df_features = df_features.select("race_id", "horse_id", "sequence")

In [ ]:
df_features = df_features.join(
    sectionals_df
        .select("race_id", "horse_id", "target")
        .dropDuplicates(["race_id", "horse_id"]), 
    on=["race_id", "horse_id"],
    how="left"
)

In [ ]:
from pyspark.sql.types import ArrayType, StructType, StructField, FloatType

# We must define the schema for the array of structs that we store in "sequence".
# If you have 6 float features, define them accordingly. For example:
feature_struct_schema = StructType([
    StructField("length_to_finish", FloatType(), True),
    StructField("sectional_time", FloatType(), True),
    StructField("running_time", FloatType(), True),
    StructField("distance_back", FloatType(), True),
    StructField("distance_ran", FloatType(), True),
    StructField("number_of_strides", FloatType(), True)
])

target_len = 38

# Python function to pad/truncate a list of structs:
def pad_sequence(seq, max_len=target_len):
    """Truncates if len(seq)>max_len, else pads with zero-like structs."""
    if seq is None:
        return []
    
    seq = list(seq)  # ensure it's mutable

    # 1) Truncate if too long
    if len(seq) > max_len:
        seq = seq[:max_len]

    # 2) Pad if too short
    while len(seq) < max_len:
        seq.append({
            "length_to_finish": 0.0,
            "sectional_time": 0.0,
            "running_time": 0.0,
            "distance_back": 0.0,
            "distance_ran": 0.0,
            "number_of_strides": 0.0
        })
    return seq

pad_sequence_udf = F.udf(lambda seq: pad_sequence(seq, target_len), 
                         ArrayType(feature_struct_schema))

df_features = df_features.withColumn(
    "padded_sequence", 
    pad_sequence_udf("sequence")
)

In [ ]:
df_features.columns

### Convert to Pandas

In [ ]:
all_pd = df_features.toPandas()

# Sort by race_date to maintain temporal order
all_pd = all_pd.sort_values("race_id").reset_index(drop=True)

### Drop NaN target

In [ ]:
# Number of rows before
old_count = len(all_pd)

# Drop rows where "target" is NaN
all_pd = all_pd.dropna(subset=["target"])

# Number of rows after
new_count = len(all_pd)

print(f"Rows before drop: {old_count}, after drop: {new_count}")
print(f"Rows dropped: {old_count - new_count}")

# Split & Normalize

## Split by Entire Race 

In [ ]:
import numpy as np
import pandas as pd

def prepare_data_3way_split_by_race(
    df,
    race_date_col=None,
    train_frac=0.70,
    val_frac=0.15,
    test_frac=0.15,
    target_len=38
):
    """
    Splits the entire dataset into (train, val, test) by race_id so that
    all horses in a race go to the same split. Then, for each subset,
    we build X arrays of shape (num_samples, target_len, 6) [or #features],
    y arrays of shape (num_samples,), and we keep an array of race_ids as well.

    Returns:
      (X_train, y_train, race_ids_train,
       X_val,   y_val,   race_ids_val,
       X_test,  y_test,  race_ids_test)
    """

    # -----------------------------
    # (A) Sort the race_ids
    # -----------------------------
    #  If you have a race_date column, use it to sort the races in chronological order
    if race_date_col and race_date_col in df.columns:
        # Make sure race_date_col is a date or something sortable
        # Then collect unique (race_id, race_date), sort by race_date
        race_date_map = (
            df[[race_date_col, "race_id"]]
            .drop_duplicates(subset=["race_id"])
            .sort_values(by=race_date_col)
        )
        race_ids_sorted = race_date_map["race_id"].tolist()
    else:
        # If no date, lexicographically sort the unique race_ids
        race_ids_sorted = sorted(df["race_id"].unique())

    n_races = len(race_ids_sorted)
    n_train = int(train_frac * n_races)
    n_val   = int(val_frac   * n_races)
    # remainder goes to test
    n_test  = n_races - (n_train + n_val)

    # slice them
    train_races = race_ids_sorted[:n_train]
    val_races   = race_ids_sorted[n_train : n_train + n_val]
    test_races  = race_ids_sorted[n_train + n_val : ]

    # -----------------------------
    # (B) Filter the main df
    # -----------------------------
    df_train = df[df["race_id"].isin(train_races)].copy()
    df_val   = df[df["race_id"].isin(val_races)].copy()
    df_test  = df[df["race_id"].isin(test_races)].copy()

    # Optionally ensure no duplicates
    df_train = df_train.drop_duplicates(subset=["race_id", "horse_id"])
    df_val   = df_val.drop_duplicates(subset=["race_id", "horse_id"])
    df_test  = df_test.drop_duplicates(subset=["race_id", "horse_id"])

    # -----------------------------
    # (C) Build arrays (X, y, race_ids) for each subset
    # -----------------------------

    def build_lstm_arrays(sub_df, target_len):
        X_list = []
        y_list = []
        rid_list = []
        hid_list = []
        
        for _, row in sub_df.iterrows():
            # This is already a list of exactly 'target_len' dicts
            seq_data = row["padded_sequence"]  

            # Convert each dict in seq_data into a numeric [6]-list
            numeric_seq = []
            for d in seq_data:
                numeric_seq.append([
                    d["length_to_finish"],
                    d["sectional_time"],
                    d["running_time"],
                    d["distance_back"],
                    d["distance_ran"],
                    d["number_of_strides"]
                ])

            X_list.append(numeric_seq)
            y_list.append(row["target"])   # or row["target"], if that’s your label
            rid_list.append(row["race_id"])
            hid_list.append(row["horse_id"])
            
        X_array = np.array(X_list, dtype=np.float32)
        y_array = np.array(y_list, dtype=np.float32)
        r_array = np.array(rid_list)
        h_array = np.array(hid_list, dtype=np.int64)
        
        return X_array, y_array, r_array, h_array

    # do it for each
    X_train, y_train, raceids_train, horseids_train = build_lstm_arrays(df_train, target_len)
    X_val,   y_val,   raceids_val,   horseids_val   = build_lstm_arrays(df_val,   target_len)
    X_test,  y_test,  raceids_test,  horseids_test  = build_lstm_arrays(df_test,  target_len)

    return (
        X_train, y_train, raceids_train, horseids_train,
        X_val,   y_val,   raceids_val,   horseids_val,
        X_test,  y_test,  raceids_test,  horseids_test
    )

## Define Helper Functions

In [ ]:
class SingleHorseDataset(Dataset):
    def __init__(self, X_array, y_array):
        """
        X_array: shape (N, seq_len, num_features)
        y_array: shape (N,)
        """
        self.X_array = X_array
        self.y_array = y_array

    def __len__(self):
        return len(self.X_array)

    def __getitem__(self, idx):
        X = self.X_array[idx]  # (seq_len, num_features)
        y = self.y_array[idx]  # scalar
        # Convert to tensors
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)
        return X_tensor, y_tensor


# SAVE YOUR SCALERS NEXT TIME

In [ ]:
def prepare_SingleHorse_data(
    X_train, y_train, race_ids_train,
    X_val,   y_val,   race_ids_val,
    X_test,  y_test,  race_ids_test,
):
    """
    1) Caps outliers (1st/99th percentile).
    2) MinMax-scales features.
    3) Builds three SingleHorseDatasets & DataLoaders (train, val, test).
    """

    def cap_outliers(data, lower=1, upper=99):
        lo = np.percentile(data, lower, axis=0)
        hi = np.percentile(data, upper, axis=0)
        return np.clip(data, lo, hi)
    
    # --- Training set ---
    num_train, seq_len, num_features = X_train.shape
    X_train_flat = X_train.reshape(-1, num_features)
    X_train_cap = cap_outliers(X_train_flat)
    scaler_X = MinMaxScaler()
    X_train_scaled_flat = scaler_X.fit_transform(X_train_cap)
    X_train_scaled = X_train_scaled_flat.reshape(num_train, seq_len, num_features)
    
    # --- Validation set ---
    num_val, seq_len_val, num_features_val = X_val.shape
    X_val_flat = X_val.reshape(-1, num_features_val)
    X_val_cap = cap_outliers(X_val_flat)
    X_val_scaled_flat = scaler_X.transform(X_val_cap)
    X_val_scaled = X_val_scaled_flat.reshape(num_val, seq_len_val, num_features_val)
    
    # --- Test set ---
    num_test, seq_len_test, num_features_test = X_test.shape
    X_test_flat = X_test.reshape(-1, num_features_test)
    X_test_cap = cap_outliers(X_test_flat)
    X_test_scaled_flat = scaler_X.transform(X_test_cap)
    X_test_scaled = X_test_scaled_flat.reshape(num_test, seq_len_test, num_features_test)
    
    ####################################
    # 1) CAP + SCALE FOR Y_TRAIN
    ####################################
    y_train_unscaled = y_train
    # Reshape to 2D so scaler can work on columns
    y_train_2d = y_train.reshape(-1, 1)

    # Cap outliers
    y_train_cap = cap_outliers(y_train_2d)

    # Fit a separate scaler for the labels
    scaler_y = MinMaxScaler()
    y_train_scaled_2d = scaler_y.fit_transform(y_train_cap)

    # Flatten back to 1D
    y_train_scaled = y_train_scaled_2d.flatten()

    ####################################
    # 2) CAP + SCALE FOR Y_VAL
    ####################################
    y_val_2d = y_val.reshape(-1, 1)
    y_val_cap = cap_outliers(y_val_2d)
    y_val_scaled_2d = scaler_y.transform(y_val_cap)
    y_val_scaled = y_val_scaled_2d.flatten()

    ####################################
    # 3) CAP + SCALE FOR Y_TEST
    ####################################
    y_test_2d = y_test.reshape(-1, 1)
    y_test_cap = cap_outliers(y_test_2d)
    y_test_scaled_2d = scaler_y.transform(y_test_cap)
    y_test_scaled = y_test_scaled_2d.flatten()
    
    ####################################
    # 4) Save Scalers
    ####################################
    save_dir="/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/models/LSTM/training_scalers_sectionals"
    try:
        os.makedirs(save_dir, exist_ok=True)
        joblib.dump(scaler_X, os.path.join(save_dir, "scaler_X.pkl"))
        joblib.dump(scaler_y, os.path.join(save_dir, "scaler_y.pkl"))
        print(f"Scalers saved to {save_dir!r}")
    except Exception as e:
        print(f"Failed to save scalers: {e!r}")

    # Build the SingleHorseDatasets
    train_dataset = SingleHorseDataset(X_train_scaled, y_train_scaled)
    val_dataset   = SingleHorseDataset(X_val_scaled,   y_val_scaled)
    test_dataset  = SingleHorseDataset(X_test_scaled,  y_test_scaled)

    # Build DataLoaders (you can pick any batch_size > 1)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False)
    test_loader  = DataLoader(test_dataset,  batch_size=16, shuffle=False)

    return (
        X_train_scaled, y_train_scaled, y_train_unscaled, scaler_X, scaler_y,
        X_val_scaled,   y_val_scaled,
        X_test_scaled,  y_test_scaled,
        scaler_X,       scaler_y,  
        train_dataset,  val_dataset,     test_dataset,
        train_loader,   val_loader,     test_loader
    )

In [ ]:
(X_train, y_train, raceids_train, horseids_train,
 X_val,   y_val,   raceids_val,   horseids_val,
 X_test,  y_test,  raceids_test,  horseids_test
)= prepare_data_3way_split_by_race(
    df=all_pd, 
    race_date_col="race_date",  # or None if you want lexicographic
    train_frac=0.70,
    val_frac=0.15,
    test_frac=0.15,
    target_len=38
)


In [ ]:
(
    X_train_scaled, y_train_scaled, y_train_unscaled, scaler_x, scaler_y,
    X_val_scaled,   y_val_scaled,
    X_test_scaled,  y_test_scaled,
    scaler_X, scaler_y,
    train_dataset, val_dataset, test_dataset,
    train_loader, val_loader, test_loader
) = prepare_SingleHorse_data(
    X_train, y_train, raceids_train,
    X_val,   y_val,   raceids_val,
    X_test,  y_test,  raceids_test
)

print("Train loader has", len(train_loader), "batches.")
print("Val loader has", len(val_loader), "batches.")
print("Test loader has", len(test_loader), "batches.")

# 🧠 Basic LSTM Model for Sequence Classification

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Device name 0:", torch.cuda.get_device_name(0))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train a Simple LSTM Regression Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HorseTimeSeriesLSTM(nn.Module):
    def __init__(
        self,
        input_size: int = 6,
        hidden_size: int = 64,
        embedding_size: int = 16,
        dropout: float = 0.0,
        num_layers: int = 1       # ← new parameter with default
    ):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,  # ← pass it here
            batch_first=True
        )
        self.dropout   = nn.Dropout(dropout)
        self.head      = nn.Linear(hidden_size, 1)
        self.projection = nn.Linear(hidden_size, embedding_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len, input_size)
        Returns:
          pred => (batch_size,) numeric regression output
          embed => (batch_size, embedding_size)
        """
        out, (h_n, c_n) = self.lstm(x)
        last_hidden    = out[:, -1, :]
        last_hidden    = self.dropout(last_hidden)
        pred           = self.head(last_hidden).squeeze(1)
        embed          = self.projection(last_hidden)
        return pred, embed

### 🧪 Mini Training Loop

In [ ]:
from tqdm.notebook import tqdm
###############################################################################
# Training Loop for One Epoch
###############################################################################
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    num_batches = 0

    for X_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        pred, embed = model(X_batch)
        loss = F.mse_loss(pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    return total_loss / num_batches if num_batches > 0 else 0.0

In [ ]:
###############################################################################
# Evaluation Loop
###############################################################################
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            pred, embed = model(X_batch)
            loss = F.mse_loss(pred, y_batch)

            total_loss += loss.item()
            num_batches += 1

    return total_loss / num_batches if num_batches > 0 else 0.0

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Device 0:", torch.cuda.get_device_name(0))
if torch.cuda.device_count() > 1:
    print("Device 1:", torch.cuda.get_device_name(1))

In [ ]:
import copy
import optuna
import torch
from torch.utils.data import DataLoader


def objective(trial):
    import torch.optim as optim

    # If you want to pick a specific GPU, or see how many you have
    gpu_count = torch.cuda.device_count()
    print("GPU Count:", gpu_count)
    device = torch.device("cuda" if gpu_count > 0 else "cpu")

    # 1) Hyperparams from Optuna
    lr = trial.suggest_float("lr", 1e-5, 5e-3, log=True)  # e.g. between 5e-4 and 5e-3
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.47, 0.9)
    num_layers = trial.suggest_int("num_layers", 1, 4)

    # 2) Build the single-horse LSTM
    model = HorseTimeSeriesLSTM(
        input_size=6,
        hidden_size=hidden_size,
        embedding_size=16,  # fixed
        dropout=dropout,
        num_layers=num_layers
    )

    # Optionally wrap in DataParallel if >1 GPU
    # Note: This requires batch_size > 1 if we want to see a speedup across GPUs.
    if gpu_count > 1:
        model = nn.DataParallel(model)

    # Move model to device
    model.to(device)

    # 3) Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 4) DataLoaders (single-horse datasets)
    #    Make sure these have batch_size > 1 if you want multi-GPU usage to help.
    #    For example, batch_size=16 or 32.
    batch_size = 128
    train_loader = DataLoader(
                    train_dataset,
                    batch_size=batch_size, 
                    shuffle=True,
                    num_workers=32,     # or 8, or more
                    pin_memory=True    # often helps on GPU
                )
    train_loader = DataLoader(
                    train_dataset,
                    batch_size=batch_size, 
                    shuffle=True,
                    num_workers=32,     # or 8, or more
                    pin_memory=True    # often helps on GPU
                )
    # 5) Early Stopping Setup
    best_val_loss = float('inf')
    best_model_state = copy.deepcopy(model.state_dict())
    patience = 5
    epochs_without_improvement = 0
    num_epochs = 50

    # 6) Main Training Loop
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, device)
        val_loss   = evaluate(model, val_loader, device)

        print(f"[Epoch {epoch+1}/{num_epochs}] train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")

        # Early stopping check
        if val_loss < best_val_loss - 1e-5:
            best_val_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        # Optuna pruning
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # 7) Load best model state and return best_val_loss
    model.load_state_dict(best_model_state)
    return best_val_loss

In [ ]:
# Function to run the Optuna study with SQLite database persistence
def run_optuna(n_trials=100):
    # Create a study with SQLite storage; it will load the study if it already exists
    study = optuna.create_study(
        study_name="horse_race_study_v1",
        storage="sqlite:///optuna_sec_lstm.db",
        direction="minimize",
        load_if_exists=True
    )
    
    # Optimize the objective function
    study.optimize(objective, n_trials=n_trials)
    
    # Print the best trial's details
    print("Best trial:")
    best_trial = study.best_trial
    print("  Validation Loss:", best_trial.value)
    print("  Best hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    
    return study

In [ ]:
# Run Optuna tuning
study = run_optuna(n_trials=100)

# Then Use the LSTM’s “Embedding” for a Ranking Model

## Build the final model


>Best trial:
>  Validation Loss: 0.002173538051654727

>  Best hyperparameters:

>    dropout: 0.7997257360718246

>    hidden_size: 128

>    lr: 0.003481825508715826

>    num_layers: 3

In [ ]:
# Retrieve the best parameters from the study
best_params = study.best_trial.params
print(best_params)

{'dropout': 0.7997257360718246, 'hidden_size': 128, 'lr': 0.003481825508715826, 'num_layers': 3}

In [ ]:
def build_combined_loader(train_dataset, val_dataset, batch_size=128, shuffle=True, 
                          pin_memory=True, num_workers=8):
    combined_dataset = ConcatDataset([train_dataset, val_dataset])
    combined_loader = DataLoader(
        combined_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=pin_memory,
        num_workers=num_workers
    )
    return combined_loader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader

class HorseTimeSeriesLSTM(nn.Module):
    def __init__(
        self,
        input_size: int = 6,
        hidden_size: int = 128,
        embedding_size: int = 16,
        dropout: float = 0.7997257360718246,
        num_layers: int = 3         # ← new argument
    ):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,    # ← pass it here
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        self.head = nn.Linear(hidden_size, 1)
        self.projection = nn.Linear(hidden_size, embedding_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len, input_size)
        Returns:
          pred => (batch_size,) numeric regression output
          embed => (batch_size, embedding_size) (optional use)
        """
        out, (h_n, c_n) = self.lstm(x)
        last_hidden = out[:, -1, :]
        last_hidden = self.dropout(last_hidden)
        pred = self.head(last_hidden).squeeze(1)
        embed = self.projection(last_hidden)
        return pred, embed

In [ ]:
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    num_batches = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        pred, _ = model(X_batch)
        loss = nn.functional.mse_loss(pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_batches += 1
        
    return total_loss / num_batches if num_batches > 0 else 0.0

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred, _ = model(X_batch)
            loss = nn.functional.mse_loss(pred, y_batch)
            total_loss += loss.item()
            num_batches += 1
            
    return total_loss / num_batches if num_batches > 0 else 0.0

In [ ]:
# load your saved scalers (so you can re-scale the test set later)
scaler_X = joblib.load(
    "/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/models/"
    "LSTM/training_scalers_sectionals/scaler_X.pkl"
)
scaler_y = joblib.load(
    "/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/models/"
    "LSTM/training_scalers_sectionals/scaler_y.pkl"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

final_model = HorseTimeSeriesLSTM(
    input_size=6,
    hidden_size=best_params["hidden_size"],
    embedding_size=16,
    dropout=0.2,                        # lower for full‐data training
    num_layers=best_params["num_layers"]
).to(device)

optimizer = optim.Adam(final_model.parameters(),
                       lr=best_params["lr"] * 0.1)


# Combine train+val
combined_loader = build_combined_loader(
    train_dataset,
    val_dataset,
    batch_size=128,  # you can pick whatever batch size you prefer
    shuffle=True,
    pin_memory=True,
    num_workers=8
)

In [ ]:
num_epochs_final = 50
for epoch in range(num_epochs_final):
    train_loss = train_one_epoch(final_model, combined_loader, optimizer, device)
    print(f"[Epoch {epoch+1}/{num_epochs_final}] combined_train_loss={train_loss:.4f}")

In [ ]:
print("Layers:", final_model.lstm.num_layers)
print("Hidden:", final_model.lstm.hidden_size)
print("Dropout:", final_model.dropout.p)

# Evaluate Model

In [ ]:
test_loss = evaluate(final_model, test_loader, device)
print(f"Final test loss = {test_loss:.4f}")

In [ ]:
import numpy as np
import torch

def evaluate_unscaled(model, test_loader, device, scaler_y):
    """
    1) Make predictions in the scaled domain.
    2) Inverse transform them to the real domain.
    3) Compute MSE/RMSE/MAE in real units.
    """
    model.eval().to(device)
    all_preds_scaled, all_labels_scaled = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            Xb = X_batch.to(device)
            pred_scaled = model(Xb)[0].cpu().numpy()
            all_preds_scaled.append(pred_scaled)
            all_labels_scaled.append(y_batch.numpy())

    all_preds_scaled  = np.concatenate(all_preds_scaled, axis=0)
    all_labels_scaled = np.concatenate(all_labels_scaled, axis=0)

    pred_unscaled   = scaler_y.inverse_transform(all_preds_scaled.reshape(-1,1)).ravel()
    truth_unscaled = scaler_y.inverse_transform(all_labels_scaled.reshape(-1,1)).ravel()

    mse  = np.mean((pred_unscaled - truth_unscaled)**2)
    rmse = np.sqrt(mse)
    mae  = np.mean(np.abs(pred_unscaled - truth_unscaled))

    print(f"Test → MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}")
    return rmse, mae, pred_unscaled, truth_unscaled

# now just call it:
rmse, mae, preds_sec, actual_sec = evaluate_unscaled(
    final_model, test_loader, device, scaler_y
)

# Update Horse Sectionals Table

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class InferenceDataset(Dataset):
    def __init__(self, X, horse_ids):
        """
        X: shape (N, seq_len, input_size) -> features for each horse
        horse_ids: shape (N,) -> integer IDs or some identifier
        """
        self.X = X
        self.horse_ids = horse_ids

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (features, horse_id)
        x_i = self.X[idx]
        h_id = self.horse_ids[idx]
        return x_i, h_id

In [ ]:
# 1) Combine them:
X_all = np.concatenate([X_train_scaled, X_val_scaled, X_test_scaled], axis=0)
X_all = X_all.astype(np.float32)  
all_horse_ids = np.concatenate([horseids_train, horseids_val, horseids_test], axis=0)

inference_dataset = InferenceDataset(X_all, all_horse_ids)
inference_loader = DataLoader(inference_dataset, batch_size=128, shuffle=False)

In [ ]:
import torch
import numpy as np
from pyspark.sql import Row

# We'll assume final_model(...) => (raw_score, embed_vec)
# raw_score: shape (batch,), embed_vec: shape (batch, 8)
final_model.eval()

projected_scores = {}   # {horse_id: raw_score}
projected_embeds = {}   # {horse_id: embedding array}

with torch.no_grad():
    for X_batch, ids_batch in inference_loader:
        X_batch = X_batch.to(device)            # shape (batch_size, seq_len, input_size)
        raw_scores, embed_vecs = final_model(X_batch)  
        # raw_scores => (batch_size,)
        # embed_vecs => (batch_size, embed_size)

        # Move them back to CPU + NumPy
        raw_scores_np = raw_scores.cpu().numpy()     # shape (batch_size,)
        embed_vecs_np = embed_vecs.cpu().numpy()     # shape (batch_size, embed_size)

        # For each item in the batch, store in the dictionaries
        for i in range(len(ids_batch)):
            horse_id = int(ids_batch[i])  # cast to int if needed
            projected_scores[horse_id] = float(raw_scores_np[i])
            projected_embeds[horse_id]   = embed_vecs_np[i]  # e.g. shape (embed_size,)
            
# 2) Min–max scale the raw scores to [0..1000]
scores_array = np.array(list(projected_scores.values()))
min_s = scores_array.min()
max_s = scores_array.max()
range_s = max_s - min_s if max_s > min_s else 1e-9

scaled_scores = {}
for horse_id, raw_score in projected_scores.items():
    normalized = (raw_score - min_s) / range_s   # [0..1]
    scaled_val = normalized * 1000.0            # [0..1000]
    scaled_scores[horse_id] = scaled_val

# 3) Build a Spark DataFrame with columns:
#  [horse_id, score, dim1, dim2, ..., dim8]

rows = []
for horse_id, sc in scaled_scores.items():
    emb = projected_embeds[horse_id]
    row_dict = {
        "horse_id": horse_id,
        "score": float(sc)
    }
    # Suppose embed size is 8 (like your example)
    for i, val in enumerate(emb):
        row_dict[f"dim{i+1}"] = float(val)

    rows.append(Row(**row_dict))

df_scores = spark.createDataFrame(rows)

# 4) Write to DB via Spark JDBC
staging_table = "horse_sectionals_lstm"

(
    df_scores.write
    .format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", staging_table)
    .option("user", jdbc_properties["user"])
    .option("driver", jdbc_properties["driver"])
    .mode("overwrite")  # or "append"
    .save()
)

# Sectionals Disribution of Scores

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_scores is your Spark DataFrame
df_scores_pandas = df_scores.select("score").toPandas()

# Plot the distribution of scores with more bins
plt.figure(figsize=(10, 6))
sns.histplot(df_scores_pandas["score"], bins=100, kde=True)  # Increase the number of bins to 100
plt.title("Distribution of Scores")
plt.xlabel("Score")
plt.ylabel("Frequency")
plt.show()

# Save Model

In [ ]:
torch.save(final_model.state_dict(), "/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/models/LSTM/sectionals_lstm_20250426.pt")

# Load Model

In [ ]:
# Retrieve the best parameters from the study
best_params = study.best_trial.params
print(best_params)

In [ ]:
model = HorseTimeSeriesLSTM(input_size=6, hidden_size=128, num_layers=3, dropout=0.2)
model.load_state_dict(torch.load("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/models/LSTM/sectionals_lstm_20250426.pt"))
model.to(device)
model.eval()

alter table horse_scores_lstm_gps
add CONSTRAINT horse_gps_lstm_id_key UNIQUE (horse_id);

CREATE INDEX idx_horse_gps_lstm ON public.horse_scores_lstm_gps USING btree (horse_id);


### MSE or MAE on Unscaled Values, e.g.m “actual seconds"

In [ ]:
# Suppose you have your trained model and your scaler_y from training:

# 1) Get predicted values on the test set (scaled):
scaled_preds = []
scaled_truth = []

for X_batch, y_batch in test_loader:
    X_batch = X_batch.to(device)
    # forward pass
    pred_batch, _ = model(X_batch)
    # Move back to numpy
    pred_batch_np = pred_batch.cpu().detach().numpy()
    scaled_preds.append(pred_batch_np)
    scaled_truth.append(y_batch.numpy())

scaled_preds = np.concatenate(scaled_preds, axis=0)
scaled_truth = np.concatenate(scaled_truth, axis=0)

# 2) Invert the transform (assuming a MinMaxScaler or StandardScaler)
unscaled_preds = scaler_y.inverse_transform(scaled_preds.reshape(-1, 1)).flatten()
unscaled_truth = scaler_y.inverse_transform(scaled_truth.reshape(-1, 1)).flatten()

# 3) Compute MSE/MAE in real units:
mse_unscaled = np.mean((unscaled_preds - unscaled_truth)**2)
mae_unscaled = np.mean(np.abs(unscaled_preds - unscaled_truth))

print("MSE in real (seconds) domain:", mse_unscaled)
print("MAE in real (seconds) domain:", mae_unscaled)

In [ ]:
import numpy as np
import torch

def evaluate_unscaled(model, test_loader, device, scaler_y):
    """
    1) Make predictions in the scaled domain over *all* batches.
    2) Inverse-transform the *entire* arrays back to real units.
    3) Compute & print MSE, RMSE, MAE in the real domain.
    """
    model.eval().to(device)
    all_preds_scaled  = []
    all_labels_scaled = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            Xb = X_batch.to(device)
            pred_scaled_t, _ = model(Xb)                # (batch_size,)
            all_preds_scaled.append(pred_scaled_t.cpu().numpy())
            all_labels_scaled.append(y_batch.numpy())

    # Concatenate across all batches
    all_preds_scaled  = np.concatenate(all_preds_scaled, axis=0)   # (N_test,)
    all_labels_scaled = np.concatenate(all_labels_scaled, axis=0)  # (N_test,)

    # Inverse-transform the *full* arrays
    pred_unscaled  = scaler_y.inverse_transform(
                        all_preds_scaled.reshape(-1,1)
                     ).ravel()
    label_unscaled = scaler_y.inverse_transform(
                        all_labels_scaled.reshape(-1,1)
                     ).ravel()

    # Compute real-domain metrics
    mse  = np.mean((pred_unscaled - label_unscaled)**2)
    rmse = np.sqrt(mse)
    mae  = np.mean(np.abs(pred_unscaled - label_unscaled))

    print(f"Test set → MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}")
    return rmse, mae, pred_unscaled, label_unscaled, all_preds_scaled

# Call it like this:
rmse, mae, preds_sec, actual_sec, scaled_preds = evaluate_unscaled(
    final_model, test_loader, device, scaler_y
)

print(f"Final RMSE in real domain: {rmse:.3f}")
print(f"Final MAE  in real domain: {mae:.3f}")

In [ ]:
rmse, mae, preds_sec, actual_sec, pred_scaled = evaluate_unscaled(final_model, test_loader, device, scaler_y)

print(f"RMSE in real domain: {rmse:.3f}")
print(f"MAE  in real domain: {mae:.3f}")

## Compare Baseline

In [ ]:
y_train_unscaled = scaler_y.inverse_transform(y_train_scaled.reshape(-1, 1)).flatten()
y_val_unscaled   = scaler_y.inverse_transform(y_val_scaled.reshape(-1, 1)).flatten()
y_test_unscaled  = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()
y_unscaled = all_pd["target"].values

In [ ]:
pred_unscaled = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

In [ ]:
import numpy as np

def compute_mean_baseline_metrics(y_train_unscaled, y_test_unscaled):
    """
    1) Compute the mean finishing time on the training set.
    2) Create baseline predictions => the same mean time for every test row.
    3) Compute MSE, RMSE, and MAE in real (seconds) domain.
    """

    # 1) Mean finishing time from training set
    train_mean_time = np.mean(y_train_unscaled)

    # 2) Baseline predictions: fill every test sample with 'train_mean_time'
    baseline_pred = np.full_like(y_test_unscaled, train_mean_time)

    # 3) Metrics
    mse  = np.mean((baseline_pred - y_test_unscaled)**2)
    rmse = np.sqrt(mse)
    mae  = np.mean(np.abs(baseline_pred - y_test_unscaled))

    print(f"[Mean Baseline]")
    print(f"  Train mean time = {train_mean_time:.3f} seconds")
    print(f"  Test MSE  = {mse:.3f}")
    print(f"  Test RMSE = {rmse:.3f}")
    print(f"  Test MAE  = {mae:.3f}")

    return mse, rmse, mae

# EXAMPLE USAGE (assuming you already have unscaled arrays):
# y_train_unscaled = ...
# y_test_unscaled  = ...
# compute_mean_baseline_metrics(y_train_unscaled, y_test_unscaled)

In [ ]:
# FIRST: invert them to get unscaled times:
y_train_unscaled = scaler_y.inverse_transform(y_train_scaled.reshape(-1,1)).ravel()
y_test_unscaled  = scaler_y.inverse_transform(y_test_scaled.reshape(-1,1)).ravel()

# THEN: call the baseline function:
compute_mean_baseline_metrics(y_train_unscaled, y_test_unscaled)